In [1]:
import pandas as pd
import pickle
from Bio import SeqIO
from sklearn.metrics import classification_report
import numpy as np

In [2]:
PATH_SAMPLE_IDS = '/home/parazit/ml_virus_host/v2.0/sample_ids/genomes_fragments_connected/' # путь к файлам, содержащим AC геномов тренировочной, валидайионной и тестовой выборок
PATH_DATA = '/home/parazit/ml_virus_host/v2.0/data/' # путь к fasta-файлам (геномы и фрагменты), таблицам с аннотцией
PATH_BLAST = "/home/parazit/ml_virus_host/v2.0/v3.0/baseline/blast/" # рабочая папка 

blast_type = "tblastx/"

Таблицы с аннотацией геномов и фрагментов

In [3]:
meta_df_genomes = pd.read_csv(PATH_DATA+"data_table.tsv", sep="\t", index_col = 0)
meta_df_800 = pd.read_csv(PATH_DATA+"data_table_800.tsv", sep="\t", index_col = 0)
meta_df_400 = pd.read_csv(PATH_DATA+"data_table_400.tsv", sep="\t", index_col = 0)

Списки AC геномов или фрагметов тренировочной, валидационной и тестовой выборок

In [4]:
sample_ids_genomes = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_genomes.pkl", "rb"))
sample_ids_800 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_800.pkl", "rb"))
sample_ids_400 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_400.pkl", "rb"))

Функция весов хозяев

In [5]:
def find_and_weight_hosts(finding, df, func_type):
    
    # выбираем функцию для подсчёта веса
    if func_type:
        function = finding.pident*finding.qcovs
    else:
        function = finding.pident
    
    # создаём таблицу, в которую записываем:
    tmp = pd.DataFrame()
    tmp["findings"] = finding.sseqid.values # AC каждой находоки
    tmp["host"] = df.loc[finding.sseqid.values].host.values # хозяина каждой находоки
    tmp["weight"] = (function/function.sum()).values # вес каждой находоки
    
    tmp.fillna(0, inplace=True)
    tmp["index"] = finding.index
    
    # в случае, если вес хозяина равен 0 (обычно при qcovs*pident значение для qcovs может равняться 0),
    # то присваиваем класс "unclassified"
    tmp.loc[tmp.weight == 0, "host"] = "unclassified" 
    tmp.set_index("index", drop=True, inplace=True)

    
    return(tmp)

Функция, анализирующая выдачу blastn

In [6]:
def blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name):

    # blastn_out - выдача blastn
    # df_query - таблица с аннотацией геномов/фрагментов, по которым проводится поиск blastn
    # df_db - таблица с аннотацией геномов, по которым создаётся база бласт
    # ids_query - AC геномов/фрагментов тестовой выборки
    # weight_type - тип веса хозяев; 0 - pident, 1 - pident*qcovs
    # save_name - название файла, который сохранит данная функция
    
    y_pred, y_true, weights = [], df_query.loc[ids_query].host.values, []


    for seq_id in ids_query: # проходимся по каждому id из тестовой выборки

        try:
            match = blastn_out.loc[seq_id] # достаём все находки бласт для данного id из тестовой выборки
            
            if type(match) == type(pd.DataFrame()): # если несколько находок, то .loc выдаёт pandas.DataFrame()
                tmp = find_and_weight_hosts(match, df_db, weight_type).groupby("host").weight.sum() # функция подсчёта весов
                predicted_host = max(zip(tmp.values, tmp.index))[1] # отбираем хозяина с наибольшим весом
                weight = max(tmp.values)
                
            if type(match) == type(pd.Series(0)): # если одна находока, то .loc выдаёт pandas.Series()
                tmp = find_and_weight_hosts(pd.DataFrame(match).T, df_db, weight_type).groupby("host").weight.sum() # функция подсчёта весов
                predicted_host = max(zip(tmp.values, tmp.index))[1] # отбираем хозяина с наибольшим весом
                weight = max(tmp.values)
            
        except KeyError: # в случае, если находок бласта нет, то присваиваем класс "unclassified"
            predicted_host = "unclassified"
            weight = 1.0
        
        # сохраняем выдачу
        y_pred.append(predicted_host), weights.append(weight)
        
    print(classification_report(y_true, y_pred, zero_division=0))
    out = pd.DataFrame(zip(ids_query, y_pred, weights), columns=["AC", "host", "weights"])
    
    out.to_csv(PATH_BLAST+"prediction/"+blast_type+save_name, sep="\t", index=0)
    return

Если переменная weight_type = 1, то вес рассчитывается по формуле: pident * qcovs
Если переменная weight_type = 0, то вес рассчитывается по формуле: pident

Важно! Классы хозяев, вес которых = 0, считаются неклассифицированными
Иногда qcovs = 0 (плохие выравнивания, стараемся избавиться от них), при этом pident отличен от 0. Находки, которые определяются с помощью pident, 

### Классификация полных геномов

Качество классификации геномов при использовании всех находок tblastx без фильтрации

In [7]:
blastn_out = pd.read_csv(PATH_BLAST + "output/tblastx/tblastx_genomes.tsv", sep="\t", index_col="qseqid")


df_query = meta_df_genomes
df_db = meta_df_genomes
ids_query = sample_ids_genomes[2]
weight_type = 1
save_name = "tblastx_prediction_genomes.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.56      0.47      0.51        95
     Mammalia       0.92      0.91      0.91       160
Viridiplantae       0.94      0.60      0.74        83
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.71       338
    macro avg       0.61      0.50      0.54       338
 weighted avg       0.83      0.71      0.76       338



Качество классификации геномов при использовании находок tblastx с длиной выравнивания 100 и более нуклеотидов

In [8]:
blastn_out = blastn_out[(blastn_out.length >= 100)]  #(blastn_out.pident >= 50.0) & 
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.72      0.40      0.51        95
     Mammalia       0.97      0.59      0.74       160
Viridiplantae       1.00      0.55      0.71        83
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.53       338
    macro avg       0.67      0.39      0.49       338
 weighted avg       0.91      0.53      0.67       338



Качество классификации геномов при использовании находок tblastx с длиной выравнивания >= 100 нуклеотидов и идентичностью (%) >= 50%

In [9]:
blastn_out = blastn_out[(blastn_out.pident >= 50.0) & (blastn_out.length >= 100)]
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.60      0.22      0.32        95
     Mammalia       1.00      0.17      0.29       160
Viridiplantae       1.00      0.23      0.37        83
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.20       338
    macro avg       0.65      0.15      0.25       338
 weighted avg       0.89      0.20      0.32       338



### Классификация фрагментов длины 800 нуклеотидов

Качество классификации фрагментов длины 800 нуклеотидов при использовании всех находок tblastx без фильтрации

In [10]:
blastn_out = pd.read_csv(PATH_BLAST + "output/tblastx/tblastx_800.tsv", sep="\t", index_col="qseqid")
 

df_query = meta_df_800
df_db = meta_df_genomes
ids_query = sample_ids_800[2]
weight_type = 1
save_name = "tblastx_prediction_800.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.56      0.23      0.33       190
     Mammalia       0.98      0.53      0.69       320
Viridiplantae       0.92      0.30      0.45       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.39       676
    macro avg       0.62      0.26      0.37       676
 weighted avg       0.85      0.39      0.53       676



Качество классификации фрагментов длины 800 нуклеотидов при использовании находок tblastx с длиной выравнивания 100 и более нуклеотидов

In [11]:
blastn_out = blastn_out[(blastn_out.length >= 100)]  #(blastn_out.pident >= 50.0) & 
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.50      0.11      0.17       190
     Mammalia       0.97      0.21      0.34       320
Viridiplantae       0.95      0.13      0.22       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.16       676
    macro avg       0.61      0.11      0.19       676
 weighted avg       0.83      0.16      0.27       676



Качество классификации фрагментов длины 800 нуклеотидов при использовании находок tblastx с длиной выравнивания >= 100 нуклеотидов и идентичностью (%) >= 50%

In [12]:
blastn_out = blastn_out[(blastn_out.pident >= 50.0) & (blastn_out.length >= 100)]
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.50      0.05      0.09       190
     Mammalia       1.00      0.05      0.10       320
Viridiplantae       1.00      0.04      0.08       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.05       676
    macro avg       0.62      0.03      0.07       676
 weighted avg       0.86      0.05      0.09       676



### Классификация фрагментов длины 400 нуклеотидов

Качество классификации фрагментов длины 400 нуклеотидов при использовании всех находок tblastx без фильтрации

In [13]:
blastn_out = pd.read_csv(PATH_BLAST + "output/tblastx/tblastx_400.tsv", sep="\t", index_col="qseqid")


df_query = meta_df_400
df_db = meta_df_genomes
ids_query = sample_ids_400[2]
weight_type = 1
save_name = "tblastx_prediction_400.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.54      0.20      0.29       190
     Mammalia       0.97      0.38      0.55       320
Viridiplantae       0.97      0.20      0.34       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.29       676
    macro avg       0.62      0.20      0.29       676
 weighted avg       0.85      0.29      0.42       676



Качество классификации фрагментов длины 400 нуклеотидов при использовании находок tblastx с длиной выравнивания 100 и более нуклеотидов

In [14]:
blastn_out = blastn_out[(blastn_out.length >= 100)]  #(blastn_out.pident >= 50.0) & 
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.56      0.08      0.14       190
     Mammalia       1.00      0.11      0.20       320
Viridiplantae       1.00      0.06      0.11       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.09       676
    macro avg       0.64      0.06      0.11       676
 weighted avg       0.88      0.09      0.16       676



Качество классификации фрагментов длины 400 нуклеотидов при использовании находок tblastx с длиной выравнивания >= 100 нуклеотидов и идентичностью (%) >= 50%

In [15]:
blastn_out = blastn_out[(blastn_out.pident >= 50.0) & (blastn_out.length >= 100)]
blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.90      0.05      0.09       190
     Mammalia       1.00      0.03      0.05       320
Viridiplantae       1.00      0.01      0.01       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.03       676
    macro avg       0.72      0.02      0.04       676
 weighted avg       0.97      0.03      0.05       676



## Результаты

In [16]:
result = pd.DataFrame()

result["Тестовая выборка"] = ["Геномы", "800нт", "400нт"]
result["Все находки tBLASTx"] = [0.76, 0.53, 0.42]
result["Находки tBLASTx с длиной выравнивания >= 100нт"] = [0.67, 0.27, 0.16]
result["Находки tBLASTx с длиной выравнивания >= 100нт и идентичностью (%) >= 50%"] = [0.32, 0.09, 0.05]

result.set_index("Тестовая выборка", inplace=True)
result

,Все находки tBLASTx,Находки tBLASTx с длиной выравнивания >= 100нт,Находки tBLASTx с длиной выравнивания >= 100нт и идентичностью (%) >= 50%
Тестовая выборка,,,
Геномы,0.76,0.67,0.32
800нт,0.53,0.27,0.09
400нт,0.42,0.16,0.05
